In [2]:
import json


# You can safely assume that `build_dataset` is correctly implemented
def build_dataset():
    data = [json.loads(x) for x in open("MLA_100k.jsonlines")]
    target = lambda x: x.get("condition")
    N = -10000
    X_train = data[:N]
    X_test = data[N:]
    y_train = [target(x) for x in X_train]
    y_test = [target(x) for x in X_test]
    for x in X_test:
        del x["condition"]
    return X_train, y_train, X_test, y_test


if __name__ == "__main__":
    print("Loading dataset...")
    # Train and test data following sklearn naming conventions
    # X_train (X_test too) is a list of dicts with information about each item.
    # y_train (y_test too) contains the labels to be predicted (new or used).
    # The label of X_train[i] is y_train[i].
    # The label of X_test[i] is y_test[i].
    X_train, y_train, X_test, y_test = build_dataset()

Loading dataset...


In [3]:
#Librerias necesarias
import pandas as pd
from pandas import json_normalize
import numpy as np
from sklearn.impute import KNNImputer
from category_encoders import CatBoostEncoder
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [8]:
#En esta funcion si hace una exploración de los datos para realizar la selección de variables. 
def preprocesamiento(data):
    
    df=pd.DataFrame(data)
    
    #En el archivo .json hay diccionarios dentro diccionarios, hago la extracción para ver que información tiene. 
    #Aquellos que estan comentados es porque por completitud o por contenido no los considero para el modelo. 
    
    #seller_address = json_normalize(df['seller_address'])
    #seller_contact = json_normalize(df['seller_contact']) 
    shipping = json_normalize(df['shipping'])
    #non_mercado_pago_payment_methods = json_normalize(df['non_mercado_pago_payment_methods']) #Lista de diccionarios
    #variations = json_normalize(df['variations']) #82600 nulos, eliminar
    #location = json_normalize(df['location']) #88000 nulos, eliminar
    #attributes = json_normalize(df['attributes']) #eliminar 78850 nulos
    #descriptions = json_normalize(df['descriptions']) #Es un campo de codigos de descripciones pero no tiene información valiosa para el problema
    #pictures = json_normalize(df['pictures']) #eliminar, no lo considero relevante para el problema
    geolocation = json_normalize(df['geolocation'])
    
    df_completo = df.join(shipping, lsuffix='_df', rsuffix='_shipping').join(geolocation, rsuffix='_geolocation')
    
    #elimino las variables que no considero aportan valor al set de datos y al objetivo del problema. 
    df_completo = df_completo.drop(columns=['seller_address',
       'seller_contact','shipping','non_mercado_pago_payment_methods', 'variations',
       'location', 'attributes','descriptions', 'pictures', 'geolocation', 'methods','free_methods'])
    
    df_completo.replace('', np.nan, inplace=True)
    
    #las siguientes variables se eliminan por completitud, faltantes por encima del 60%
    df_completo = df_completo.drop(columns=['warranty',
       'sub_status','deal_ids','listing_source', 'coverage_areas',
       'official_store_id', 'differential_pricing','original_price', 'video_id', 'catalog_product_id', 'subtitle','tags_shipping',
                                       'dimensions'])
    #site_id y international_delivery_mode tiene el mismo valor para todos los registros
    #price tiene outliers o errores. Se filtra todo menor a 9.000.000 y mayor que 0
    #thumbnail, secure_thumbnail y permalink son url que concatenan informacio de otros campos como id, description y que no tienen informacion relevante para el problema

    df_completo = df_completo.drop(columns=['site_id', 'international_delivery_mode', 'thumbnail', 'secure_thumbnail', 'permalink'])
    #Fechas: last_updated, date_created, stop_time, start_time contienen informacion sobre publicación, actualizacion o detencion de la publicación,no son determinantes del estado del articulo.
    df_completo = df_completo.drop(columns=['last_updated', 'date_created', 'stop_time', 'start_time'])
    #Se elimina 'Condition' del set de entrenamiento puesto que esta en y_train
    if 'condition' in df_completo.columns:
        # Verifica si la columna existe en el DataFrame y la elimina si es así
        df_completo = df_completo.drop(columns=['condition'])
        
    #estos codigos pueden tener información valiosa para determinar si es nuevo o usado
    df_completo['parent_item_id'] = df_completo['parent_item_id'].str.replace('^MLA', '', regex=True)
    df_completo['category_id'] = df_completo['category_id'].str.replace('^MLA', '', regex=True)
    df_completo['id'] = df_completo['id'].str.replace('^MLA', '', regex=True)
    
    #Cambiar el tipo de dato de category_id y id
    df_completo['category_id'] = df_completo['category_id'].astype('int64')
    df_completo['id'] = df_completo['id'].astype('int64')
    
    #Reemplazar los nulos en tags_df por el valor mas frecuente que es ['dragged_bids_and_visits']
    df_completo['tags_df'] = df_completo['tags_df'].fillna("['dragged_bids_and_visits']")
    
    # Extraer el contenido de la lista en 'tags_df' y asignarlo a una nueva columna 'tags'
    df_completo['tags'] = df_completo['tags_df'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

    # Si deseas eliminar la columna 'tags_df' original, puedes hacerlo así
    df_completo.drop(columns=['tags_df'], inplace=True)
    
    return df_completo   

   

In [9]:
X_train=preprocesamiento(X_train)

In [10]:
X_test=preprocesamiento(X_test)

In [11]:
#imputacion de valores faltantes en tags_df, parent_item_id, Latitud y Longitud (estas ultimas en una experimentacion futura se pueden evaluar)
# Construimos el modelo
imputer = KNNImputer(n_neighbors=16, weights="uniform")

# Ajustamos el modelo e imputamos los missing values en xtrain
imputer.fit(X_train[["parent_item_id"]])
X_train["parent_item_id"] = imputer.transform(X_train[["parent_item_id"]]).ravel()
# Imputamos los missing values en xtest con el modelo de X_train
X_test["parent_item_id"] = imputer.transform(X_test[["parent_item_id"]]).ravel()

# Ajustamos el modelo e imputamos los missing values, se tratan los datos como float, son el el 5% del total en Xtrain
imputer.fit(X_train[["latitude"]])
X_train["latitude"] = imputer.transform(X_train[["latitude"]]).ravel()
X_test["latitude"] = imputer.transform(X_test[["latitude"]]).ravel()

imputer.fit(X_train[["longitude"]])
X_train["longitude"] = imputer.transform(X_train[["longitude"]]).ravel()
X_test["longitude"] = imputer.transform(X_test[["longitude"]]).ravel()

In [13]:
#se realiza codificacion de la variable objetivo

# Inicializar el codificador LabelEncoder
label_encoder = LabelEncoder()

# Ajustar el codificador a los datos de la variable objetivo (y_train)
y_train_encoded = label_encoder.fit_transform(y_train)

# Ahora y_train_encoded contiene los valores codificados de la variable objetivo


In [15]:
#Se usa el codificador CatBoost ya que funciona asignando valores numéricos a cada categoría en una característica en 
#función de su relación con la variable objetivo

# Seleccionar las columnas categóricas a codificar
categorical_features = ["listing_type_id", "buying_mode", "tags", "currency_id", "status", "mode"]

# Inicializar el codificador CatBoost
encoder = CatBoostEncoder()

# Ajustar el codificador a los datos
encoder.fit(X_train[categorical_features], y_train_encoded)  # Utiliza squeeze() para convertir DataFrame a Serie

# Codificar las variables categóricas en xtrain
X_train_encoded = encoder.transform(X_train[categorical_features])
# Codificar las variables categóricas en xtest
X_test_encoded = encoder.transform(X_test[categorical_features])

# Concatenar las variables codificadas con el DataFrame original
X_train_encoded = pd.concat([X_train.drop(columns=categorical_features), X_train_encoded], axis=1)
X_test_encoded = pd.concat([X_test.drop(columns=categorical_features), X_test_encoded], axis=1)

In [21]:
#Se realiza la vectorización del texto del titulo de la publicacion para poder usarlos en el modelo
#Primero se limpia el texto para quitar carecteres y numeros y dejar solo texto limpio

# Función para limpiar el texto
def limpiar_texto(texto):
# Convertir a minúsculas
    texto = texto.lower()
    # Remover caracteres especiales y números
    texto = re.sub(r'[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ]+', ' ', texto)
    # Remover espacios adicionales
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto


# Limpiar el texto en la columna 'title'
X_train_encoded['title_limpio'] = X_train_encoded['title'].apply(limpiar_texto)
X_test_encoded['title_limpio'] = X_test_encoded['title'].apply(limpiar_texto)

# Inicializar el vectorizador TF-IDF
# Se dejan las 1000 palabras mas frecuentes en todo los textos.
vectorizer = TfidfVectorizer(max_features=1000)

# Ajustar y transformar los datos de texto limpios
vectorizer.fit(X_train_encoded['title_limpio'])
X_tfidf_train = vectorizer.transform(X_train_encoded['title_limpio'])
X_tfidf_test = vectorizer.transform(X_test_encoded['title_limpio'])

# Convertir los resultados a un DataFrame de pandas
X_tfidf_train = pd.DataFrame(X_tfidf_train.toarray(), columns=vectorizer.get_feature_names_out())
X_tfidf_test = pd.DataFrame(X_tfidf_test.toarray(), columns=vectorizer.get_feature_names_out())

#se une la vectorizacion al dataframe completo
X_train_final = X_train_encoded.join(X_tfidf_train, lsuffix='_df', rsuffix='_tfidf')
X_test_final = X_test_encoded.join(X_tfidf_test, lsuffix='_df', rsuffix='_tfidf')

X_train_final = X_train_final.drop(columns=['title'])
X_test_final = X_test_final.drop(columns=['title'])

#Cambiar el tipo de dato de category_id y id
X_train_final['category_id'] = X_train_final['category_id'].astype('int64')
X_train_final['id'] = X_train_final['id'].astype('int64')

X_test_final['category_id'] = X_test_final['category_id'].astype('int64')
X_test_final['id'] = X_test_final['id'].astype('int64')

In [37]:
X_train_final = X_train_final.drop(columns=['title_limpio'])
X_test_final = X_test_final.drop(columns=['title_limpio'])

In [36]:
pd.set_option('display.max_columns',None)
X_train_final.head()

,base_price,seller_id,price,parent_item_id,category_id,id,accepts_mercadopago,automatic_relist,initial_quantity,sold_quantity,available_quantity,local_pick_up,free_shipping,latitude,longitude,listing_type_id,buying_mode,tags,currency_id,status,mode,title_limpio,abeja,accesorios,aceite,acero,acondicionado,acrilico,adaptador,adidas,adorno,afiche,ag,agua,air,aire,aires,al,alberto,album,aleman,alemania,alfa,alfombra,algodon,algodón,all,alma,alquiler,alta,alto,aluminio,ambientes,america,american,americano,amor,amortiguador,amortiguadores,amplificador,ana,anal,and,angel,anillo,animal,anteojos,antigua,antiguas,antiguo,antiguos,antonio,aplique,apple,araña,arg,argentina,argentino,argentinos,aros,art,arte,artesanal,artesonido,as,asiento,at,atlas,audi,audio,auricular,auriculares,auto,autos,azul,año,años,baby,bafle,bajo,banda,bandeja,bar,barbie,barra,base,bateria,batería,batman,baul,baño,beatles,bebe,belgrano,benz,biblioteca,bicicleta,big,billete,black,blackberry,blanca,blanco,blister,blu,blue,bluetooth,bmw,bobina,boca,body,bolsa,bolso,bomba,book,bora,borges,bosch,botas,botella,box,brasil,bronce,buen,buenos,buzo,by,caba,caballito,cabezal,cable,cables,cadena,cafe,caja,cajas,cajones,calada,calidad,cama,camara,camino,camion,camisa,camisas,camiseta,campeon,campera,campo,candy,canon,cap,capital,capot,car,carcasa,carga,cargador,cargo,carlos,cars,cartas,cartel,carteles,cartera,cartucho,casa,casco,case,casi,casio,cassette,caña,caño,cc,cd,cds,celular,central,centro,ceramica,cerrado,cerveza,chaleco,chapa,chevrolet,china,chocolate,ciencia,ciencias,cierre,cilindro,cine,cinta,circular,citroen,ciudad,civic,classic,clio,club,cm,cocina,col,cola,colchon,coleccion,colección,colgante,collar,color,colores,combo,comedor,como,completa,completas,completo,con,confeccion,conjunto,consolador,consultar,control,corazon,cordoba,core,correa,corsa,corta,corte,cortina,corto,corven,cover,crema,cristal,cruz,cuadro,cuadros,cubierta,cubre,cuchillo,cuentos,cuero,cuerpo,cuerpos,cultura,cuna,cámara,da,dama,daniel,david,db,dc,ddr,de,deco,decoracion,del,delantera,delantero,delanteros,departamento,deportivo,der,derecha,derecho,desde,di,dia,diario,diccionario,diesel,digital,dije,direccion,disco,discos,diseño,diseños,disfraz,disney,display,distribucion,dj,doble,dodge,don,dorado,dos,dr,dragon,ds,dual,duna,dura,dvd,día,ebook,eco,ecosport,ed,edicion,ediciones,edición,edition,editorial,eduardo,el,electrica,electrico,embrague,en,english,enrique,entre,entrega,envio,envios,envío,envíos,epson,equipo,era,es,escala,escape,escort,escritorio,españa,español,especial,espejo,espiral,espirales,estado,estampada,estampado,estilo,estuche,estudio,etc,europa,ex,excelente,exterior,extra,fabrica,falcon,faro,farol,fender,ferrari,fiat,fiesta,figura,figurita,figuritas,film,filtro,firmado,flash,flex,flores,focus,for,ford,foto,fotos,fox,frances,francia,francisco,freno,frente,fric,fuego,fuente,full,funcionando,funda,futbol,galaxy,garantia,garcia,gas,gb,gel,general,george,gj,glass,gnc,gol,gold,golf,goma,gps,gr,grafico,gran,grand,grande,grandes,gratis,green,gris,grs,grupo,gráfico,gt,gtia,guantes,guardabarro,guerra,guia,guitarra,hasta,hd,hdmi,heladera,hermosa,hermoso,hierro,high,hilo,hilux,historia,hojas,hombre,hombres,honda,hot,hp,ideal,ii,iii,imp,impecable,impecables,imperdible,importada,importadas,importado,importados,impresora,imprimible,in,inalambrico,independiente,industrial,infantil,infantiles,ingles,inglesa,inglés,inoxidable,insignia,intel,interior,ipad,iphone,iron,isbn,italia,italiano,izquierdo,jack,james,jardin,jazz,jean,jeans,jeep,jgo,john,jorge,jose,josé,juan,juego,juegos,juguete,julio,kangoo,kawasaki,kg,king,kit,kits,km,la,lamina,lampara,lana,larga,largo,las,laser,lata,lcd,le,led,lente,lentes,leon,les,letras,lg,liberado,libre,libro,libros,linea,liquido,lista,literatura,litros,live,llanta,llave,llavero,lo,local,lona,lopez,lorenzo,los,lote,love,lp,lpr,lt,lts,luces,luis,luna,luz,lycra,mac,macizo,made,madera,madre,magic,malla,man,manga,mangas,manija,mano,manoenpez,manos,manual,manuel,maquina,mar,marca,ma

In [53]:
X_Train_final = X_train_final.to_numpy()
X_Test_final = X_test_final.to_numpy()

In [30]:
# Ajustar el codificador a los datos de la variable objetivo (y_test)
y_test_encoded = label_encoder.fit_transform(y_test)

In [66]:
from xgboost.sklearn import XGBClassifier
import xgboost as xgb


train = xgb.DMatrix(X_Train_final, y_train_encoded)
test  = xgb.DMatrix(X_Test_final, y_test_encoded)

max_rounds = 3
#Estamos ante un problema binario, por lo tanto el objetivo es binario y usamos la regresion logistica para hacer la funcion de separació de las categorias. 
#para esta primera corrida dejo una profundidad pequeña para eficiencia computacional y evitar sobreajuste.
#sin embargo el learning rate si lo dejo pequeño para que el algoritmo explore minusiosamente. 
params = {
    'objective':'binary:logistic',
    'eval_metric':['auc', 'error'],
    'max_depth': 3,
    'learning_rate': 0.1
}

for i in range(3):
    print(f'xgb.train - ROUND ({i})')
    params['seed']=np.random.randint(100)
    xgb.train(params, train, max_rounds, evals=[(test, 'validation_0')], verbose_eval=True)

print()
    
for i in range(3):
    print(f'XGBClassifier - ROUND ({i})')
    params['seed']=np.random.randint(100)
    clf = XGBClassifier(n_estimators=max_rounds, **params)
    clf.fit(X_Train_final, y_train_encoded, eval_set=[(X_Test_final, y_test_encoded)], verbose=True);

xgb.train - ROUND (0)
[0]	validation_0-auc:0.88829	validation_0-error:0.18550
[1]	validation_0-auc:0.88831	validation_0-error:0.18550
[2]	validation_0-auc:0.89205	validation_0-error:0.18550
xgb.train - ROUND (1)
[0]	validation_0-auc:0.88829	validation_0-error:0.18550
[1]	validation_0-auc:0.88831	validation_0-error:0.18550
[2]	validation_0-auc:0.89205	validation_0-error:0.18550
xgb.train - ROUND (2)
[0]	validation_0-auc:0.88829	validation_0-error:0.18550
[1]	validation_0-auc:0.88831	validation_0-error:0.18550
[2]	validation_0-auc:0.89205	validation_0-error:0.18550

XGBClassifier - ROUND (0)
[0]	validation_0-auc:0.88829	validation_0-error:0.18550
[1]	validation_0-auc:0.88831	validation_0-error:0.18550
[2]	validation_0-auc:0.89205	validation_0-error:0.18550
XGBClassifier - ROUND (1)
[0]	validation_0-auc:0.88829	validation_0-error:0.18550
[1]	validation_0-auc:0.88831	validation_0-error:0.18550
[2]	validation_0-auc:0.89205	validation_0-error:0.18550
XGBClassifier - ROUND (2)
[0]	validation_

In [55]:
#creacion y entrenamiento del modelo
from xgboost import XGBClassifier
from sklearn import metrics
eval_metric = ['auc','error']
clf = XGBClassifier(booster = 'gbtree', eval_metric = eval_metric, seed=123)

In [58]:
#para esta busqueda aleatoria incluyo objetivo hinge ya que quiero explorar como performa con este objetivo que no da las probabilidades si no tambien tiene en cuenta el hiperplano de clasificación. 
objective = ['binary:logistic','binary:hinge'] #
# Maxima profundidad, valores exploratorios
max_depth = [5, 10, 20, 25]
#se prueban estos valores para encontrar el optimo. 
eta = [0.1, 0.2, 0.3]

random_grid = {'objective': objective,
                'eta': eta,
               'max_depth': max_depth}

In [ ]:
#implmentacion de busqueda aleatoria
from sklearn.model_selection import RandomizedSearchCV
xgb_random = RandomizedSearchCV(estimator = clf,
                               param_distributions = random_grid,
                               n_iter = 12, cv = 3,
                               verbose=2,
                               random_state=42, n_jobs = -1)
xgb_random.fit(X_Train_final, y_train_encoded)

In [ ]:
#se sacan los mejores parametros para entrenar el modelo con ellos. ({'objective': 'binary:logistic', 'max_depth': 20, 'eta': 0.3})
mejores_parametros = xgb_random.best_params_

In [ ]:
clf_mejorado = XGBClassifier(**mejores_parametros)

In [63]:
clf_mejorado.fit(X_Train_final, y_train_encoded)
y_pred = clf.predict(X_Test_final)
metrics.f1_score(y_pred, y_test_encoded), metrics.accuracy_score(y_pred, y_test_encoded)

In [54]:
from sklearn.metrics import roc_auc_score

y_true = y_test_encoded  # Etiquetas reales
y_scores = clf.predict_proba(X_Test_final)[:, 1]  # Puntuaciones de predicción

auc = roc_auc_score(y_true, y_scores)

print(f"AUC: {auc}")


(0.8814033577270771, 0.8898)

In [44]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10,
                            seed=123)